# Hyperparameter Search

[TOC]

In [1]:
import onmt
from onmt.trainer import Trainer
from onmt.utils.logging import logger
from onmt.utils.loss import build_loss_compute
import onmt.opts as opts
from onmt.inputters.inputter import build_dataset_iter, \
    load_old_vocab, old_style_vocab
from onmt.model_builder import build_model
from onmt.utils.optimizers import Optimizer
from onmt.utils.misc import set_random_seed
from onmt.models import build_model_saver
from onmt.utils.logging import init_logger, logger
from onmt.utils.misc import split_corpus
from onmt.translate.translator import build_translator

import itertools
import configargparse
import os
from itertools import chain
import torch
import subprocess, shlex, re, uuid, hashlib, json, gc, shutil, sys
from pprint import pprint
from copy import deepcopy

## Utils

In [2]:
class ArgumentHelper:
    def build(self, arg_dict):
        arg_total = self.base_arg_dict.copy()
        arg_total.update(arg_dict)
        return arg_total
    
    @staticmethod
    def save_args(arg_dict, filename):
        with open(filename + '.txt', 'w') as f:
            json.dump(arg_dict, f, indent=4)
    
    @staticmethod
    def convert_dict_to_arg_array(arg_dict):
        arr = []
        for k, v in arg_dict.items():
            arr.append('-' + k)
            if v is not None:
                arr.append(str(v))
        return arr

In [3]:
def cleanup():
    torch.cuda.empty_cache()
    gc.collect()

In [4]:
def build_translate_args(model_id):
    return {
        'src': 'data/nmt15/test_en',
        'tgt': 'data/nmt15/test_vi',
        'out': '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt',
        'replace_unk': None,
        'gpu': '0',
        'batch_size': '16'
    }

def build_translate_args_reverse(model_id):
    return {
        'src': 'data/nmt15/test_vi',
        'tgt': 'data/nmt15/test_en',
        'out': '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt',
        'replace_unk': None,
        'gpu': '0',
        'batch_size': '16'
    }

In [5]:
def clean_path_with_no_checkpoint(model_path, log_path):
    useless_file = []
    available_model = set()
    useless_log = []
    for root, dirs, files in os.walk(model_path):
        path = root.split(os.sep)
        for file in files:
            absolute_filename = os.path.join(root, file)
            model_id = re.findall(r'opennmt_(.+)_step_\d+\.pt', file)
            if len(model_id) != 0:
                available_model.add(model_id[0])
    
    for root, dirs, files in os.walk(model_path):
        path = root.split(os.sep)
        for file in files:
            absolute_filename = os.path.join(root, file)
            model_id = re.findall(r'opennmt_(.+).txt', file)
            if len(model_id) != 0 and model_id[0] not in available_model:
                useless_file.append(absolute_filename)
    
    dirs = os.listdir(log_path)
    for d in dirs:
        model_id = re.findall(r'opennmt_(.+)', d)
        if len(model_id) != 0 and model_id[0] not in available_model:
            useless_log.append(os.path.join(log_path, d))
    
#     print(available_model)
#     print(useless_file)
#     print(useless_log)
    for f in useless_file:
        os.remove(f)
    for f in useless_log:
        print(f)
        !rm -rf $f

# clean_path_with_no_checkpoint('/mnt/drive-2t/model', '/mnt/drive-2t/log')

In [6]:
def evaluate_human(model_id):
    model_args = '/mnt/drive-2t/model/opennmt_' + model_id + '.txt'
    !cat $model_args
    pred = '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt'
    ref = 'data/nmt15/test_en'
    print()
    print(bleu_evaluate(pred, ref), bleu_evaluate(pred, ref, '0.25 0.25 0.25 0.25'))
    pred_head = !head $pred
    ref_head = !head $ref
    pairs = list(zip(pred_head, ref_head))
    for pair in pairs:
        print(pair[0])
        print()
        print(pair[1])
        print('-------------')

## BLEU

In [7]:
class BleuHelper:
    
    def __init__(self):
        self.python_interpreter = '/home/ray/.conda/envs/dl/bin/python'
        self.bleu_script = '/home/ray/Smooth_BLEU/bleu.py'
        self.nltk_script = '/home/ray/Smooth_BLEU/nltk_bleu.py'
        self.bleu_pred = '/home/ray/OpenNMT-py/data/nmt15/pred.txt'
        self.bleu_reference = '/home/ray/OpenNMT-py/data/nmt15/test_vi'
        self.bleu_weight = '0.25 0.25 0.25'
    
    def smooth_bleu_evaluate(self, pred, reference, weight='0.25 0.25 0.25'):
        p = subprocess.Popen([self.python_interpreter, self.bleu_script,
                              '-t', pred,
                              '-r', reference,
                              '-w', weight], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        p.wait()
        out, err = p.communicate()
        if p.returncode != 0:
            print(err)
            raise Exception()
        out = out.decode()
        err = err.decode()
        bleu_score = re.findall(r'BLEU = ([1-9\.]+)', err)
        return float(bleu_score[0])

    def nltk_bleu_evaluate(self, pred, reference, weight='0.25 0.25 0.25'):
        p = subprocess.Popen([self.python_interpreter, self.nltk_script,
                              '-t', pred,
                              '-r', reference,
                              '-w', weight], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        p.wait()
        out, err = p.communicate()
        if p.returncode != 0:
            print(err)
            raise Exception()
        out = out.decode()
        err = err.decode()
        bleu_score = re.findall(r'BLEU = ([1-9\.]+)', err)
        return float(bleu_score[0])

bleu_helper = BleuHelper()
def bleu_evaluate(pred, reference, weight='0.25 0.25 0.25'):
    smooth = bleu_helper.smooth_bleu_evaluate(pred, reference, weight)
#     nltk_ = bleu_helper.nltk_bleu_evaluate(pred, reference, weight)
#     return smooth, nltk_
    return smooth

## Searchable Hyperparameters

In [8]:
# searchable hyperparameters
class SearchableHyperparameters:
    
    def __init__(self):
        embedding_size = ['64', '128', '256', '512']
        encoder_type = ['rnn', 'brnn', 'mean', 'transformer']
        decoder_type = encoder_type
        layers = ['1', '2', '3', '4', '5', '6']
        rnn_size = ['64', '128', '256', '512']
        rnn_type = ['LSTM', 'GRU', 'SRU']
        global_attention = ['dot', 'general', 'mlp']
        self_attention_type = ['scaled_dot', 'average']
        heads = [2, 4, 6, 8, 10]
        transformer_ff = [1024, 2048, 4096]

        normalization = ['sents', 'tokens']
        valid_steps = 1000
        train_steps = 10000
        dropout = 0.1

## Custom Trainer Class

Features:
1. Early stop supported.
2. Custom validation evaluation functions.(TODO)
3. Custom callbacks. (TODO)

In [9]:
class MyTrainer(Trainer):
    def train(self,
              train_iter,
              train_steps,
              save_checkpoint_steps=5000,
              valid_iter=None,
              valid_steps=10000,
              early_stop_round=50000,
              early_stop_threshold=1):
        self.last_model_saver = None
        self.last_step = None
        self.last_moving_average = None
        if valid_iter is None:
            logger.info('Start training loop without validation...')
        else:
            logger.info('Start training loop and validate every %d steps...',
                        valid_steps)

        total_stats = onmt.utils.Statistics()
        report_stats = onmt.utils.Statistics()
        self._start_report_manager(start_time=total_stats.start_time)
        
        valid_early_stop_loss = []
        last_accent_valid_acc = float('-inf')
        decent_round = 0
        for i, (batches, normalization) in enumerate(
                self._accum_batches(train_iter)):
            if i == 0:
                print(i, 'Testing GPU memory capability with batch shape:', batches[0].src[0].shape, batches[0].tgt.shape)
            step = self.optim.training_step
            if decent_round == 0:  # 如果没有处于下降过程，则保存前一个model，如果当前step下降，则一直保存这个model
                self.last_model_saver = deepcopy(self.model_saver)
                self.last_step = step
                self.last_moving_average = deepcopy(self.moving_average)

            self._gradient_accumulation(
                batches, normalization, total_stats,
                report_stats)

            if self.average_decay > 0 and i % self.average_every == 0:
                self._update_average(step)
                            
            # 向tensorboard输出loss
            if step % self.report_manager.report_every == 0:
                tensorboard_writer = self.report_manager.tensorboard_writer
                tensorboard_writer.add_scalar('train/loss', report_stats.loss, step)

            report_stats = self._maybe_report_training(
                step, train_steps,
                self.optim.learning_rate(),
                report_stats)

            
            if valid_iter is not None and step % valid_steps == 0:
                print('last accent valid acc:', last_accent_valid_acc)
                if step == valid_steps:
                    valid_stats = self.validate(
                        valid_iter, verbose=1, moving_average=self.moving_average)
                else:
                    valid_stats = self.validate(
                        valid_iter, moving_average=self.moving_average)
                valid_stats = self._maybe_gather_stats(valid_stats)
                tensorboard_writer.add_scalar('valid/loss', valid_stats.loss, step)
#                 valid_acc = valid_stats.accuracy()
                valid_acc = -valid_stats.loss
                
#                 self._report_step(self.optim.learning_rate(),
#                                   step, valid_stats=valid_stats)
#                 if last_accent_valid_acc + early_stop_threshold > valid_acc:
#                     print('meet early stop condition, prev best loss:', last_accent_valid_acc, 'current loss:', valid_acc)
#                     if self.model_saver is not None:
#                         self.model_saver.save(step, moving_average=self.moving_average)
#                     return total_stats
                
#                 last_accent_valid_acc = valid_acc
                
                if valid_acc < last_accent_valid_acc + early_stop_threshold:  # 开始下降
                    if decent_round == 0:  # 保存最后一个上升model
                        print('Save last accent model with acc:', last_accent_valid_acc)
                    decent_round += 1
                    print('Decent round:', decent_round)
                else:
                    decent_round = 0
                    last_accent_valid_acc = valid_acc

                self._report_step(self.optim.learning_rate(),
                                  step, valid_stats=valid_stats)
                if decent_round == early_stop_round:  # 停止
                    if self.last_model_saver is not None:
                        print('meet early stop condition, prev best loss:', last_accent_valid_acc, 'current loss:', valid_acc)
                        self.last_model_saver.save(self.last_step, moving_average=self.last_moving_average)
                    return total_stats

#                 if len(valid_early_stop_loss) == early_stop_round:
#                     # 条件为当前valid loss要超过前n个valid loss最好的那个加threshold (X)
#                     # 应该允许valid loss暂时下降，但不能持续下降超过n个valid round
#                     # 如果持续下降，回到最好的model
#                     if valid_loss - max(valid_early_stop_loss) < early_stop_threshold:
#                         print('meet early stop condition, prev best loss:', max(valid_early_stop_loss), 'current loss:', valid_loss)
#                         if self.model_saver is not None:
#                             self.model_saver.save(step, moving_average=self.moving_average)
#                             self._report_step(self.optim.learning_rate(),
#                                               step, valid_stats=valid_stats)
#                         return total_stats
#                     else:
#                         valid_early_stop_loss.pop(0)
#                         valid_early_stop_loss.append(valid_loss)

            if (self.model_saver is not None
                    and (save_checkpoint_steps != 0
                         and step % save_checkpoint_steps == 0)):
                self.model_saver.save(step, moving_average=self.moving_average)

            if train_steps > 0 and step >= train_steps:
                break

        if self.model_saver is not None:
            self.model_saver.save(step, moving_average=self.moving_average)
        return total_stats
    
    def validate(self, valid_iter, verbose=0, moving_average=None):
        if moving_average:
            valid_model = deepcopy(self.model)
            for avg, param in zip(self.moving_average,
                                  valid_model.parameters()):
                param.data = avg.data.half() if self.model_dtype == "fp16" \
                    else avg.data
        else:
            valid_model = self.model

        # Set model in validating mode.
        valid_model.eval()

        with torch.no_grad():
            stats = onmt.utils.Statistics()

            for batch in valid_iter:
#                 print('Testing GPU memory capability with batch shape:', batch.src[0].shape, batch.tgt.shape)
                src, src_lengths = batch.src if isinstance(batch.src, tuple) \
                                   else (batch.src, None)
                tgt = batch.tgt

                # F-prop through the model.
                outputs, attns = valid_model(src, tgt, src_lengths)

                # Compute loss.
                _, batch_stats = self.valid_loss(batch, outputs, attns)

                # Update statistics.
                stats.update(batch_stats)

        if moving_average:
            del valid_model
        else:
            # Set model back to training mode.
            valid_model.train()

        return stats

In [10]:
def build_trainer(opt, device_id, model, fields, optim, model_saver=None):
    tgt_field = fields['tgt'][0][1].base_field
    train_loss = build_loss_compute(model, tgt_field, opt)
    valid_loss = build_loss_compute(
        model, tgt_field, opt, train=False)

    trunc_size = opt.truncated_decoder  # Badly named...
    shard_size = opt.max_generator_batches if opt.model_dtype == 'fp32' else 0
    norm_method = opt.normalization
    grad_accum_count = opt.accum_count
    n_gpu = opt.world_size
    average_decay = opt.average_decay
    average_every = opt.average_every
    if device_id >= 0:
        gpu_rank = opt.gpu_ranks[device_id]
    else:
        gpu_rank = 0
        n_gpu = 0
    gpu_verbose_level = opt.gpu_verbose_level

    report_manager = onmt.utils.build_report_manager(opt)
    trainer = MyTrainer(model, train_loss, valid_loss, optim, trunc_size,
                        shard_size, norm_method,
                        grad_accum_count, n_gpu, gpu_rank,
                        gpu_verbose_level, report_manager,
                        model_saver=model_saver if gpu_rank == 0 else None,
                        average_decay=average_decay,
                        average_every=average_every,
                        model_dtype=opt.model_dtype)
    return trainer

## Training Entry

In [11]:
def _check_save_model_path(opt):
    save_model_path = os.path.abspath(opt.save_model)
    model_dirname = os.path.dirname(save_model_path)
    if not os.path.exists(model_dirname):
        os.makedirs(model_dirname)


def _tally_parameters(model):
    enc = 0
    dec = 0
    for name, param in model.named_parameters():
        if 'encoder' in name:
            enc += param.nelement()
        else:
            dec += param.nelement()
    return enc + dec, enc, dec


def training_opt_postprocessing(opt, device_id):
    if opt.word_vec_size != -1:
        opt.src_word_vec_size = opt.word_vec_size
        opt.tgt_word_vec_size = opt.word_vec_size

    if opt.layers != -1:
        opt.enc_layers = opt.layers
        opt.dec_layers = opt.layers

    if opt.rnn_size != -1:
        opt.enc_rnn_size = opt.rnn_size
        opt.dec_rnn_size = opt.rnn_size

        # this check is here because audio allows the encoder and decoder to
        # be different sizes, but other model types do not yet
        same_size = opt.enc_rnn_size == opt.dec_rnn_size
        assert opt.model_type == 'audio' or same_size, \
            "The encoder and decoder rnns must be the same size for now"

    opt.brnn = opt.encoder_type == "brnn"

    assert opt.rnn_type != "SRU" or opt.gpu_ranks, \
        "Using SRU requires -gpu_ranks set."

    if torch.cuda.is_available() and not opt.gpu_ranks:
        logger.info("WARNING: You have a CUDA device, \
                    should run with -gpu_ranks")

    if device_id >= 0:
        torch.cuda.set_device(device_id)
    set_random_seed(opt.seed, device_id >= 0)

    return opt


def train_main(opt, device_id):
    opt = training_opt_postprocessing(opt, device_id)
    init_logger(opt.log_file)
    # Load checkpoint if we resume from a previous training.
    if opt.train_from:
        logger.info('Loading checkpoint from %s' % opt.train_from)
        checkpoint = torch.load(opt.train_from,
                                map_location=lambda storage, loc: storage)

        # Load default opts values then overwrite it with opts from
        # the checkpoint. It's usefull in order to re-train a model
        # after adding a new option (not set in checkpoint)
        dummy_parser = configargparse.ArgumentParser()
        opts.model_opts(dummy_parser)
        default_opt = dummy_parser.parse_known_args([])[0]

        model_opt = default_opt
        model_opt.__dict__.update(checkpoint['opt'].__dict__)
        logger.info('Loading vocab from checkpoint at %s.' % opt.train_from)
        vocab = checkpoint['vocab']
    else:
        checkpoint = None
        model_opt = opt
        vocab = torch.load(opt.data + '.vocab.pt')

    # check for code where vocab is saved instead of fields
    # (in the future this will be done in a smarter way)
    if old_style_vocab(vocab):
        data_type = opt.model_type
        fields = load_old_vocab(vocab, data_type, dynamic_dict=opt.copy_attn)
    else:
        fields = vocab

    # Report src and tgt vocab sizes, including for features
    for side in ['src', 'tgt']:
        for name, f in fields[side]:
            try:
                f_iter = iter(f)
            except TypeError:
                f_iter = [(name, f)]
            for sn, sf in f_iter:
                if sf.use_vocab:
                    logger.info(' * %s vocab size = %d' % (sn, len(sf.vocab)))

    # Build model.
    model = build_model(model_opt, opt, fields, checkpoint)
    n_params, enc, dec = _tally_parameters(model)
    logger.info('encoder: %d' % enc)
    logger.info('decoder: %d' % dec)
    logger.info('* number of parameters: %d' % n_params)
    _check_save_model_path(opt)

    # Build optimizer.
    optim = Optimizer.from_opt(model, opt, checkpoint=checkpoint)

    # Build model saver
    model_saver = build_model_saver(model_opt, opt, model, fields, optim)

    trainer = build_trainer(
        opt, device_id, model, fields, optim, model_saver=model_saver)

    # this line is kind of a temporary kludge because different objects expect
    # fields to have a different structure
    dataset_fields = dict(chain.from_iterable(fields.values()))

    train_iter = build_dataset_iter("train", dataset_fields, opt)
    valid_iter = build_dataset_iter(
        "valid", dataset_fields, opt, is_train=False)

    if len(opt.gpu_ranks):
        logger.info('Starting training on GPU: %s' % opt.gpu_ranks)
    else:
        logger.info('Starting training on CPU, could be very slow')
    train_steps = opt.train_steps
    if opt.single_pass and train_steps > 0:
        logger.warning("Option single_pass is enabled, ignoring train_steps.")
        train_steps = 0
    trainer.train(
        train_iter,
        train_steps,
        save_checkpoint_steps=opt.save_checkpoint_steps,
        valid_iter=valid_iter,
        valid_steps=opt.valid_steps,
        early_stop_round=opt.early_stop_round,
        early_stop_threshold=opt.early_stop_threshold)

    if opt.tensorboard:
        trainer.report_manager.tensorboard_writer.close()

## Home-made Runner Class

Features:
1. Automatic hyperparameters search. (TODO)
2. Reusable.

In [12]:
class BaseRunner:
    
    def __init__(self, data_path):
        self.data_path = data_path
        self.model_id = None
    
    def run(self, option, model_id=None, train_from=None):
        model_id = model_id if model_id is not None else BaseRunner.generate_id()
        self.model_id = model_id
        self.prepare_save_file()
        arg_dict = {
            'data': self.data_path,
            'save_model': self.model_path,
            'gpu_ranks': 0,
            'tensorboard': None,
            'tensorboard_log_dir': self.log_path + '/log'
        }
        arg_dict.update(option)
        if train_from is not None:
            arg_dict['train_from'] = train_from
        ArgumentHelper.save_args(arg_dict, self.model_path)
        parser = configargparse.ArgumentParser(
            description='train.py',
            formatter_class=configargparse.ArgumentDefaultsHelpFormatter)

        opts.add_md_help_argument(parser)
        opts.model_opts(parser)
        opts.train_opts(parser)
        parser.add_argument('-early_stop_round', type=int, default=10000, help='Early stop round')
        parser.add_argument('-early_stop_threshold', type=float, default=1, help='Early stop threshold')

        opt = parser.parse_args(ArgumentHelper.convert_dict_to_arg_array(arg_dict))
        train_main(opt, 0)

    def translate_main(self, opt, logger, model_id):
        translator = build_translator(opt, report_score=True)
        src_shards = split_corpus(opt.src, opt.shard_size)
        tgt_shards = split_corpus(opt.tgt, opt.shard_size) \
            if opt.tgt is not None else [None]*opt.shard_size
        shard_pairs = zip(src_shards, tgt_shards)

        for i, (src_shard, tgt_shard) in enumerate(shard_pairs):
            logger.info("Translating shard %d." % i)
            translator.translate(
                src=src_shard,
                tgt=tgt_shard,
                src_dir=opt.src_dir,
                batch_size=opt.batch_size,
                attn_debug=opt.attn_debug
            )

    def translate(self, args, model_id=None):
        model_id = model_id if model_id is not None else self.model_id
        if model_id is None:
            raise Exception()
        self.model_id = model_id
        self.prepare_save_file()
        args['model'] = self.model_path + '_step_' + str(BaseRunner.get_latest_model(
                                                        self.model_dir,
                                                        self.model_prefix,
                                                        self.model_id)) + '.pt'
        parser = configargparse.ArgumentParser(
            description='translate.py',
            config_file_parser_class=configargparse.YAMLConfigFileParser,
            formatter_class=configargparse.ArgumentDefaultsHelpFormatter)
        opts.config_opts(parser)
        opts.add_md_help_argument(parser)
        opts.translate_opts(parser)

        opt = parser.parse_args(ArgumentHelper.convert_dict_to_arg_array(args))
        logger = init_logger(opt.log_file)
        self.translate_main(opt, logger, model_id)

    @staticmethod
    def generate_id():
        return hashlib.md5(str(uuid.uuid4()).encode()).hexdigest()
    
    @staticmethod
    def get_latest_model(model_dir, model_prefix, model_id):
        all_steps = []
        for root, dirs, files in os.walk(model_dir):
            path = root.split(os.sep)
            for file in files:
                absolute_filename = os.path.join(root, file)
                if model_id in absolute_filename:
                    steps = re.findall(model_prefix + model_id + '_step_(\d+)\.pt', absolute_filename)
                    if len(steps) > 0:
                        all_steps.append(int(steps[0]))
        return max(all_steps)

    def prepare_save_file(self):
        model_id = self.model_id
        model_dir = '/mnt/drive-2t/model/'
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model_prefix = 'opennmt_'
        model_path = model_dir + model_prefix + model_id
        log_dir = '/mnt/drive-2t/log/'
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        log_prefix = model_prefix
        log_path = log_dir + log_prefix + model_id
        if not os.path.exists(log_path):
            os.mkdir(log_path)
        self.model_path = model_path
        self.log_path = log_path
        self.model_prefix = model_prefix
        self.model_dir = model_dir


runner = BaseRunner('data/nmt15')

In [13]:
reverse_runner = BaseRunner('data/nmt15-reverse')

In [14]:
torch.cuda.memory_allocated(0) / 1024 / 1024

0.0

In [15]:
torch.cuda.memory_cached(0) / 1024 / 1024 

0.0

In [16]:
torch.cuda.max_memory_cached(0) / 1024 / 1024

0.0

In [17]:
all_vars = {k: v for k, v in globals().items() if not k.startswith('_') and k not in ['In', 'Out']}

In [18]:
var_type = {k: str(type(v)) for k, v in all_vars.items() if 'function' not in str(type(v)) and
                                                            'module' not in str(type(v))}
var_type

{'get_ipython': "<class 'method'>",
 'exit': "<class 'IPython.core.autocall.ZMQExitAutocall'>",
 'quit': "<class 'IPython.core.autocall.ZMQExitAutocall'>",
 'Trainer': "<class 'type'>",
 'logger': "<class 'logging.RootLogger'>",
 'Optimizer': "<class 'type'>",
 'chain': "<class 'type'>",
 'ArgumentHelper': "<class 'type'>",
 'BleuHelper': "<class 'type'>",
 'bleu_helper': "<class '__main__.BleuHelper'>",
 'SearchableHyperparameters': "<class 'type'>",
 'MyTrainer': "<class 'type'>",
 'BaseRunner': "<class 'type'>",
 'runner': "<class '__main__.BaseRunner'>",
 'reverse_runner': "<class '__main__.BaseRunner'>"}

In [19]:
del reverse_runner
gc.collect()
reverse_runner = BaseRunner('data/nmt15-reverse')

## Example #0: Preprocessing

In [29]:
# %%bash
# python preprocess.py \
#      -train_src data/nmt15/train_vi \
#      -train_tgt data/nmt15/train_en \
#      -valid_src data/nmt15/valid_vi \
#      -valid_tgt data/nmt15/valid_en \
#      -save_data data/nmt15-reverse \
#      --src_seq_length 200 \
#      --tgt_seq_length 200 \
#      --src_words_min_frequency 3 \
#      --tgt_words_min_frequency 3

## Example #1: Train A Basic LSTM Network

In [16]:
model_args = {
    'encoder_type': 'brnn',
    'rnn_type': 'GRU'
}
train_args = {
    'batch_size': 64,
    'train_steps': 20000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 5,
    'early_stop_threshold': 0.01,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)
reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-16 19:26:39,631 INFO]  * src vocab size = 8870
[2019-02-16 19:26:39,632 INFO]  * tgt vocab size = 20071
[2019-02-16 19:26:39,633 INFO] Building model...
[2019-02-16 19:26:43,167 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 500, padding_idx=1)
        )
      )
    )
    (rnn): GRU(500, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedGRU(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): GRUCell(1000, 500)
        (1): GRUCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out): L

last accent valid acc: -inf


[2019-02-16 19:29:13,092 INFO] Validation perplexity: 75.9304
[2019-02-16 19:29:13,093 INFO] Validation accuracy: 32.9474
[2019-02-16 19:29:42,160 INFO] Step 600/20000; acc:  31.85; ppl: 88.06; xent: 4.48; lr: 0.00100; 3590/3064 tok/s;    179 sec
[2019-02-16 19:30:11,635 INFO] Step 700/20000; acc:  32.60; ppl: 79.42; xent: 4.37; lr: 0.00100; 5425/4624 tok/s;    208 sec
[2019-02-16 19:30:41,019 INFO] Step 800/20000; acc:  34.85; ppl: 65.90; xent: 4.19; lr: 0.00100; 5312/4527 tok/s;    238 sec
[2019-02-16 19:31:11,013 INFO] Step 900/20000; acc:  35.54; ppl: 60.60; xent: 4.10; lr: 0.00100; 5210/4449 tok/s;    268 sec
[2019-02-16 19:31:41,459 INFO] Step 1000/20000; acc:  35.98; ppl: 57.13; xent: 4.05; lr: 0.00100; 5233/4462 tok/s;    298 sec
[2019-02-16 19:31:41,569 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654


last accent valid acc: 32.94736280636874


[2019-02-16 19:31:57,461 INFO] Validation perplexity: 42.9186
[2019-02-16 19:31:57,462 INFO] Validation accuracy: 38.7609
[2019-02-16 19:32:28,021 INFO] Step 1100/20000; acc:  36.74; ppl: 53.38; xent: 3.98; lr: 0.00100; 3430/2899 tok/s;    345 sec
[2019-02-16 19:32:59,141 INFO] Step 1200/20000; acc:  37.88; ppl: 48.85; xent: 3.89; lr: 0.00100; 5121/4354 tok/s;    376 sec
[2019-02-16 19:33:30,448 INFO] Step 1300/20000; acc:  38.31; ppl: 45.87; xent: 3.83; lr: 0.00100; 5035/4287 tok/s;    407 sec
[2019-02-16 19:34:01,939 INFO] Step 1400/20000; acc:  38.50; ppl: 44.79; xent: 3.80; lr: 0.00100; 4983/4254 tok/s;    439 sec
[2019-02-16 19:34:34,977 INFO] Loading dataset from data/nmt15-reverse.train.0.pt, number of examples: 95844
[2019-02-16 19:34:35,735 INFO] Step 1500/20000; acc:  39.53; ppl: 40.56; xent: 3.70; lr: 0.00100; 4622/3948 tok/s;    472 sec
[2019-02-16 19:34:35,833 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654


...

[2019-02-16 20:23:05,842 INFO] Step 12100/20000; acc:  47.35; ppl: 15.85; xent: 2.76; lr: 0.00100; 6208/5251 tok/s;   3383 sec
[2019-02-16 20:23:15,557 INFO] Step 12200/20000; acc:  49.83; ppl: 12.82; xent: 2.55; lr: 0.00100; 14673/12697 tok/s;   3392 sec
[2019-02-16 20:23:26,640 INFO] Step 12300/20000; acc:  48.20; ppl: 14.50; xent: 2.67; lr: 0.00100; 14648/12510 tok/s;   3403 sec
[2019-02-16 20:23:37,437 INFO] Step 12400/20000; acc:  48.31; ppl: 14.79; xent: 2.69; lr: 0.00100; 14591/12360 tok/s;   3414 sec
[2019-02-16 20:23:48,124 INFO] Step 12500/20000; acc:  48.29; ppl: 14.55; xent: 2.68; lr: 0.00100; 15101/12862 tok/s;   3425 sec
[2019-02-16 20:23:48,223 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654


last accent valid acc: 47.30724439268538


[2019-02-16 20:24:04,205 INFO] Validation perplexity: 19.3363
[2019-02-16 20:24:04,205 INFO] Validation accuracy: 47.2952


Decent round: 5
meet early stop condition, prev best loss: 47.30724439268538 current loss: 47.29524300585845


AttributeError: 'NMTModel' object has no attribute 'optim'

In [15]:
train_args['valid_batch'] = 64
reverse_runner.run(train_args)

[2019-02-15 20:13:46,283 INFO]  * src vocab size = 8870
[2019-02-15 20:13:46,284 INFO]  * tgt vocab size = 20071
[2019-02-15 20:13:46,284 INFO] Building model...
[2019-02-15 20:13:49,837 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out): Linear(in_feature

meet early stop condition, prev loss: 52.793656155822454 current loss: 53.52796323130673


In [16]:
reverse_runner.model_id

'18afdcd77652c3d69836a58403953bac'

## Example #2: Train A Transformer

In [18]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'transformer',
    'decoder_type': 'transformer',
    'layers': 6,
    'position_encoding': None,
    'max_generator_batches': 2,
    'dropout': 0.1,
    'enc_rnn_size': 512,
    'dec_rnn_size': 512,
    'src_word_vec_size': 512,
    'tgt_word_vec_size': 512,
    'accum_count': 4
}
train_args = {
    'batch_size': 16,
    'train_steps': 100000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 200,
    'valid_batch': 16,
    'early_stop_round': 10,
    'early_stop_threshold': 0,
    'report_every': 100
}
total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-17 13:47:46,400 INFO]  * src vocab size = 8870
[2019-02-17 13:47:46,402 INFO]  * tgt vocab size = 20071
[2019-02-17 13:47:46,403 INFO] Building model...
[2019-02-17 13:47:50,630 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax()
          (dropout): Dropout(p=0.1)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
       

0 Testing GPU memory capability with batch shape: torch.Size([200, 16, 1]) torch.Size([190, 16, 1])


[2019-02-17 13:49:28,193 INFO] Step 100/100000; loss: 2621894.360388 acc:   9.87; ppl: 1468.88; xent: 7.29; lr: 0.00100; 4701/3692 tok/s;     97 sec
[2019-02-17 13:50:44,865 INFO] Step 200/100000; loss: 1312841.478624 acc:  19.83; ppl: 207.88; xent: 5.34; lr: 0.00100; 3932/3208 tok/s;    174 sec
[2019-02-17 13:50:44,973 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654


last accent valid acc: -inf


[2019-02-17 13:51:07,936 INFO] Validation perplexity: 169.087
[2019-02-17 13:51:07,937 INFO] Validation accuracy: 17.9167
[2019-02-17 13:52:20,121 INFO] Step 300/100000; loss: 1025134.557983 acc:  22.01; ppl: 155.75; xent: 5.05; lr: 0.00100; 2590/2132 tok/s;    269 sec
[2019-02-17 13:53:28,372 INFO] Step 400/100000; loss: 846213.914762 acc:  23.66; ppl: 125.23; xent: 4.83; lr: 0.00100; 3088/2567 tok/s;    338 sec
[2019-02-17 13:53:28,476 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654


...

last accent valid acc: -977666.166809082


[2019-02-17 14:05:03,622 INFO] Validation perplexity: 88.2704
[2019-02-17 14:05:03,622 INFO] Validation accuracy: 27.9317


Decent round: 2


[2019-02-17 14:05:32,770 INFO] Loading dataset from data/nmt15-reverse.train.0.pt, number of examples: 95844


RuntimeError: CUDA out of memory. Tried to allocate 23.62 MiB (GPU 0; 5.76 GiB total capacity; 3.86 GiB already allocated; 35.06 MiB free; 228.79 MiB cached)

In [17]:
{k: v in globals().items() if k 

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import onmt\nfrom onmt.trainer import Trainer\nfrom onmt.utils.logging import logger\nfrom onmt.utils.loss import build_loss_compute\nimport onmt.opts as opts\nfrom onmt.inputters.inputter import build_dataset_iter, \\\n    load_old_vocab, old_style_vocab\nfrom onmt.model_builder import build_model\nfrom onmt.utils.optimizers import Optimizer\nfrom onmt.utils.misc import set_random_seed\nfrom onmt.models import build_model_saver\nfrom onmt.utils.logging import init_logger, logger\nfrom onmt.utils.misc import split_corpus\nfrom onmt.translate.translator import build_translator\n\nimport itertools\nimport configargparse\nimport os\nfrom itertools import chain\nimport torch\nimport subprocess, shlex, re, uuid, has

## Example #3: Make Prediction

In [29]:
model_id = '832c1d8d66c693bce372445f692b0099'

translate_args = {
    'src': 'data/nmt15/test_en',
    'tgt': 'data/nmt15/test_vi',
    'out': '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt',
    'replace_unk': None,
    'gpu': '0',
    'batch_size': '16'
}

runner.translate(translate_args, model_id)

[2019-02-15 17:15:34,546 WARNING] FP16 is experimental, the generated checkpoints may be incompatible with a future version
[2019-02-15 17:15:34,564 INFO] Translating shard 0.
[2019-02-15 17:22:55,574 INFO] Translating shard 1.


PRED AVG SCORE: -0.6200, PRED PPL: 1.8589
GOLD AVG SCORE: -10.4237, GOLD PPL: 33648.8349


[2019-02-15 17:30:52,361 INFO] Translating shard 2.


PRED AVG SCORE: -0.6197, PRED PPL: 1.8584
GOLD AVG SCORE: -10.4420, GOLD PPL: 34270.6062
PRED AVG SCORE: -0.6319, PRED PPL: 1.8812
GOLD AVG SCORE: -10.4421, GOLD PPL: 34270.9004


In [38]:
class MyTrainer(Trainer):
    def train(self,
              train_iter,
              train_steps,
              save_checkpoint_steps=5000,
              valid_iter=None,
              valid_steps=10000,
              early_stop_round=50000,
              early_stop_threshold=1):
        if valid_iter is None:
            logger.info('Start training loop without validation...')
        else:
            logger.info('Start training loop and validate every %d steps...',
                        valid_steps)

        total_stats = onmt.utils.Statistics()
        report_stats = onmt.utils.Statistics()
        self._start_report_manager(start_time=total_stats.start_time)
        
        valid_early_stop_loss = []
        for i, (batches, normalization) in enumerate(
                self._accum_batches(train_iter)):
            step = self.optim.training_step

            self._gradient_accumulation(
                batches, normalization, total_stats,
                report_stats)

            if self.average_decay > 0 and i % self.average_every == 0:
                self._update_average(step)

            report_stats = self._maybe_report_training(
                step, train_steps,
                self.optim.learning_rate(),
                report_stats)

            if valid_iter is not None and step % valid_steps == 0:
                valid_stats = self.validate(
                    valid_iter, moving_average=self.moving_average)
                valid_stats = self._maybe_gather_stats(valid_stats)
                valid_loss = valid_stats.accuracy()
#                 valid_early_stop_loss.append(valid_loss)
                if len(valid_early_stop_loss) == early_stop_round:
                    # 条件为当前valid loss要超过前n个valid loss最好的那个加threshold
                    if valid_loss - max(valid_early_stop_loss) < early_stop_threshold:
                        print('meet early stop condition, prev best loss:', max(valid_early_stop_loss), 'current loss:', valid_loss)
                        if self.model_saver is not None:
                            self.model_saver.save(step, moving_average=self.moving_average)
                        return total_stats
                    else:
                        valid_early_stop_loss.pop(0)
                        valid_early_stop_loss
                self._report_step(self.optim.learning_rate(),
                                  step, valid_stats=valid_stats)

            if (self.model_saver is not None
                    and (save_checkpoint_steps != 0
                         and step % save_checkpoint_steps == 0)):
                self.model_saver.save(step, moving_average=self.moving_average)

            if train_steps > 0 and step >= train_steps:
                break

        if self.model_saver is not None:
            self.model_saver.save(step, moving_average=self.moving_average)
        return total_stats

(36.37, 2.0)

该model(transformer)的valid loss = 51.17859672702835, last_train_acc = 58.17

In [34]:
model_id = '5cb980256df787cdd84b166f65503553'
runner.translate(build_translate_args(model_id), model_id)

[2019-02-15 19:46:30,266 INFO] Translating shard 0.
[2019-02-15 19:48:39,104 INFO] Translating shard 1.


PRED AVG SCORE: -0.4451, PRED PPL: 1.5607
GOLD AVG SCORE: -11.8941, GOLD PPL: 146406.9002


[2019-02-15 19:50:56,516 INFO] Translating shard 2.


PRED AVG SCORE: -0.4401, PRED PPL: 1.5529
GOLD AVG SCORE: -11.8323, GOLD PPL: 137626.8257
PRED AVG SCORE: -0.4524, PRED PPL: 1.5722
GOLD AVG SCORE: -11.7261, GOLD PPL: 123763.2465


In [37]:
pred = '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt'
ref = 'data/nmt15/test_vi'
bleu_evaluate(pred, ref), bleu_evaluate(pred, ref, '0.25 0.25 0.25 0.25')

(4.0, 24.16)

LSTM model valid loss = 53.67019634197286, last_train_acc = 67.98

In [15]:
model_id = '18afdcd77652c3d69836a58403953bac'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-15 20:47:26,898 INFO] Translating shard 0.
[2019-02-15 20:48:58,846 INFO] Translating shard 1.


PRED AVG SCORE: -0.6594, PRED PPL: 1.9336
GOLD AVG SCORE: -11.0811, GOLD PPL: 64930.4421


[2019-02-15 20:50:33,734 INFO] Translating shard 2.


PRED AVG SCORE: -0.6708, PRED PPL: 1.9557
GOLD AVG SCORE: -11.0883, GOLD PPL: 65401.4528
PRED AVG SCORE: -0.6949, PRED PPL: 2.0034
GOLD AVG SCORE: -11.0109, GOLD PPL: 60531.6975


In [16]:
pred = '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt'
ref = 'data/nmt15/test_en'
bleu_evaluate(pred, ref), bleu_evaluate(pred, ref, '0.25 0.25 0.25 0.25')

(35.73, 21.21)

In [20]:
from pprint import pprint
pred_head = !head $pred
ref_head = !head $ref
pairs = list(zip(pred_head, ref_head))
for pair in pairs:
    print(pair[0])
    print()
    print(pair[1])
    print('-------------')

Now , it &apos;s time that I ended up , and as I mentioned before , of course , I &apos;ve got a lot of other data if you &apos;re interested , but I just want to give this basic idea about communication to its language , and the potential of that possibility .

Now I just want to wrap up , and as I was mentioning earlier of course I have a lot of other data if you &apos;re interested , but I just wanted to give this sort of basic idea of being able to communicate with the brain in its language , and the potential power of being able to do that .
-------------
This is very different from the artificial device that you have to bridge from the brain to a device . Here we have to bridge the world from the external brain to a device .

So it &apos;s different from the motor prosthetics where you &apos;re communicating from the brain to a device . Here we have to communicate from the outside world into the brain and be understood , and be understood by the brain .
-------------
And then the

In [21]:
evaluate_human('18afdcd77652c3d69836a58403953bac')

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_18afdcd77652c3d69836a58403953bac",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_18afdcd77652c3d69836a58403953bac/log",
    "batch_size": 128,
    "train_steps": 20000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 64,
    "early_stop_round": 5,
    "early_stop_threshold": 1,
    "report_every": 100
}35.73 21.21
Now , it &apos;s time that I ended up , and as I mentioned before , of course , I &apos;ve got a lot of other data if you &apos;re interested , but I just want to give this basic idea about communication to its language , and the potential of that possibility .

Now I just want to wrap up , and as I was mentioning earlier of course I have a lot of other data if you &apos;re interested , but I just wanted to give this sort of basic idea of being able to communicate with the brain in its language , and

In [18]:
model_id = '5384edb1bec7646d9b0008074304d283'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 16
reverse_runner.translate(args, model_id)

[2019-02-16 11:08:55,819 WARNING] FP16 is experimental, the generated checkpoints may be incompatible with a future version
[2019-02-16 11:08:55,847 INFO] Translating shard 0.
[2019-02-16 11:17:00,743 INFO] Translating shard 1.


PRED AVG SCORE: -0.8561, PRED PPL: 2.3540
GOLD AVG SCORE: -10.7383, GOLD PPL: 46086.8161


[2019-02-16 11:24:55,976 INFO] Translating shard 2.


PRED AVG SCORE: -0.8727, PRED PPL: 2.3933
GOLD AVG SCORE: -10.7712, GOLD PPL: 47628.2838
PRED AVG SCORE: -0.8991, PRED PPL: 2.4575
GOLD AVG SCORE: -10.7426, GOLD PPL: 46286.4810


In [19]:
pred = '/mnt/drive-2t/pred/opennmt_' + model_id + '.txt'
ref = 'data/nmt15/test_en'
bleu_evaluate(pred, ref), bleu_evaluate(pred, ref, '0.25 0.25 0.25 0.25')

(31.22, 17.0)

In [22]:
from pprint import pprint
pred_head = !head $pred
ref_head = !head $ref
pairs = list(zip(pred_head, ref_head))
for pair in pairs:
    print(pair[0].replace('&apos;', '\''))
    print()
    print(pair[1].replace('&apos;', '\''))
    print('-------------')

Now , at the end of the end of the conclusion , I 've mentioned before , I 've got a lot of other data , but if you want to give me a sense of the idea of the brain , it 's ability to interact with the potential language , and now , it 's right now .

Now I just want to wrap up , and as I was mentioning earlier of course I have a lot of other data if you 're interested , but I just wanted to give this sort of basic idea of being able to communicate with the brain in its language , and the potential power of being able to do that .
-------------
This is very different with artificial biology , and you have to communicate from the brain to the brain . Here we have to be able to communicate north through the world , and understand the brain .

So it 's different from the motor prosthetics where you 're communicating from the brain to a device . Here we have to communicate from the outside world into the brain and be understood , and be understood by the brain .
-------------
And then fina

## Example #4: GRU

In [15]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'rnn',
    'decoder_type': 'rnn',
    'rnn_type': 'GRU',
    'layers': 3,
    'max_generator_batches': 2,
    'dropout': 0.1,
    'enc_rnn_size': 512,
    'dec_rnn_size': 512,
    'src_word_vec_size': 512,
    'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 64,
    'train_steps': 20000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 5,
    'early_stop_threshold': 0.1,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-16 13:22:25,039 INFO]  * src vocab size = 8870
[2019-02-16 13:22:25,042 INFO]  * tgt vocab size = 20071
[2019-02-16 13:22:25,042 INFO] Building model...
[2019-02-16 13:22:28,630 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 512, padding_idx=1)
        )
      )
    )
    (rnn): GRU(512, 512, num_layers=3, dropout=0.1)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 512, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.1)
    (rnn): StackedGRU(
      (dropout): Dropout(p=0.1)
      (layers): ModuleList(
        (0): GRUCell(1024, 512)
        (1): GRUCell(512, 512)
        (2): GRUCell(512, 512)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=512, out_features=512, bias=False)
      (lin

meet early stop condition, prev loss: 36.793585036493106 current loss: 37.37898601616187


'990547e9b5cc3ef3ab8df6728e6f8669'

In [15]:
model_id = '990547e9b5cc3ef3ab8df6728e6f8669'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-16 13:42:42,908 INFO] Translating shard 0.
[2019-02-16 13:44:19,325 INFO] Translating shard 1.


PRED AVG SCORE: -1.1561, PRED PPL: 3.1776
GOLD AVG SCORE: -10.0213, GOLD PPL: 22499.8106


[2019-02-16 13:45:57,107 INFO] Translating shard 2.


PRED AVG SCORE: -1.1661, PRED PPL: 3.2096
GOLD AVG SCORE: -10.0651, GOLD PPL: 23508.4893
PRED AVG SCORE: -1.1886, PRED PPL: 3.2823
GOLD AVG SCORE: -10.0625, GOLD PPL: 23446.4471


In [23]:
evaluate_human('990547e9b5cc3ef3ab8df6728e6f8669')

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_990547e9b5cc3ef3ab8df6728e6f8669",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_990547e9b5cc3ef3ab8df6728e6f8669/log",
    "model_dtype": "fp32",
    "encoder_type": "rnn",
    "decoder_type": "rnn",
    "rnn_type": "GRU",
    "layers": 3,
    "max_generator_batches": 2,
    "dropout": 0.1,
    "enc_rnn_size": 512,
    "dec_rnn_size": 512,
    "src_word_vec_size": 512,
    "tgt_word_vec_size": 512,
    "batch_size": 64,
    "train_steps": 20000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 64,
    "early_stop_round": 5,
    "early_stop_threshold": 0.1,
    "report_every": 100
}
23.3 11.89
Now , before I go back to this time , I &apos;m going to show you , but I &apos;m going to talk about this , but it &apos;s a combination of time , and if you &apos;re talking about it , it &apos;s the idea of the human b

In [16]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'rnn',
    'decoder_type': 'rnn',
    'rnn_type': 'GRU',
#     'layers': 3,
#     'max_generator_batches': 2,
    'dropout': 0.3,
#     'enc_rnn_size': 512,
#     'dec_rnn_size': 512,
#     'src_word_vec_size': 512,
#     'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 64,
    'train_steps': 20000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 5,
    'early_stop_threshold': 0.1,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-16 13:57:27,389 INFO]  * src vocab size = 8870
[2019-02-16 13:57:27,390 INFO]  * tgt vocab size = 20071
[2019-02-16 13:57:27,392 INFO] Building model...
[2019-02-16 13:57:30,951 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 500, padding_idx=1)
        )
      )
    )
    (rnn): GRU(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedGRU(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): GRUCell(1000, 500)
        (1): GRUCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out): Linear(in_features=10

meet early stop condition, prev best loss: 44.19043978415284 current loss: 40.965622694177995


'f40ce12cd75f5edc1a96a6e26f46ed4e'

In [18]:
model_id = 'f40ce12cd75f5edc1a96a6e26f46ed4e'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-16 14:09:21,216 INFO] Translating shard 0.
[2019-02-16 14:10:34,134 INFO] Translating shard 1.


PRED AVG SCORE: -1.0952, PRED PPL: 2.9897
GOLD AVG SCORE: -9.4220, GOLD PPL: 12357.4480


[2019-02-16 14:11:48,342 INFO] Translating shard 2.


PRED AVG SCORE: -1.1152, PRED PPL: 3.0503
GOLD AVG SCORE: -9.4861, GOLD PPL: 13175.3264
PRED AVG SCORE: -1.1401, PRED PPL: 3.1272
GOLD AVG SCORE: -9.5160, GOLD PPL: 13575.0565


In [19]:
evaluate_human('f40ce12cd75f5edc1a96a6e26f46ed4e')

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_f40ce12cd75f5edc1a96a6e26f46ed4e",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_f40ce12cd75f5edc1a96a6e26f46ed4e/log",
    "model_dtype": "fp32",
    "encoder_type": "rnn",
    "decoder_type": "rnn",
    "rnn_type": "GRU",
    "dropout": 0.3,
    "batch_size": 64,
    "train_steps": 20000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 64,
    "early_stop_round": 5,
    "early_stop_threshold": 0.1,
    "report_every": 100
}
21.52 12.44
Now , this time I ended up , and I mentioned earlier , of course , of course , but I wanted to give this idea of this idea about its body .

Now I just want to wrap up , and as I was mentioning earlier of course I have a lot of other data if you &apos;re interested , but I just wanted to give this sort of basic idea of being able to communicate with the brain in its language ,

In [16]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'rnn',
    'decoder_type': 'rnn',
#     'rnn_type': 'GRU',
#     'layers': 3,
#     'max_generator_batches': 2,
    'dropout': 0.3,
#     'enc_rnn_size': 512,
#     'dec_rnn_size': 512,
#     'src_word_vec_size': 512,
#     'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 64,
    'train_steps': 20000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 5,
    'early_stop_threshold': 0.1,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-16 14:23:42,127 INFO]  * src vocab size = 8870
[2019-02-16 14:23:42,128 INFO]  * tgt vocab size = 20071
[2019-02-16 14:23:42,130 INFO] Building model...
[2019-02-16 14:23:45,681 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out): Linear(in_feature

meet early stop condition, prev best loss: 53.28260154506743 current loss: 53.23859646003538


'22846dcbc32b2f51a395010cde5d7f85'

In [17]:
model_id = '22846dcbc32b2f51a395010cde5d7f85'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-16 14:44:18,390 INFO] Translating shard 0.
[2019-02-16 14:45:53,548 INFO] Translating shard 1.


PRED AVG SCORE: -0.7247, PRED PPL: 2.0640
GOLD AVG SCORE: -10.3031, GOLD PPL: 29825.0073


[2019-02-16 14:47:31,194 INFO] Translating shard 2.


PRED AVG SCORE: -0.7346, PRED PPL: 2.0846
GOLD AVG SCORE: -10.2928, GOLD PPL: 29518.2979
PRED AVG SCORE: -0.7631, PRED PPL: 2.1449
GOLD AVG SCORE: -10.2362, GOLD PPL: 27895.0392


In [19]:
evaluate_human('22846dcbc32b2f51a395010cde5d7f85')

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_22846dcbc32b2f51a395010cde5d7f85",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_22846dcbc32b2f51a395010cde5d7f85/log",
    "model_dtype": "fp32",
    "encoder_type": "rnn",
    "decoder_type": "rnn",
    "dropout": 0.3,
    "batch_size": 64,
    "train_steps": 20000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 64,
    "early_stop_round": 5,
    "early_stop_threshold": 0.1,
    "report_every": 100
}
35.56 21.0
Now , it &apos;s time that I ended up , and as I mentioned earlier , I had a lot of other data if you &apos;re interested in , but I just wanted to give this basic idea about communication with the brain of it , and the potential of that ability to be the potential of that ability .

Now I just want to wrap up , and as I was mentioning earlier of course I have a lot of other data if you &apos;re inte

In [16]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'brnn',
    'decoder_type': 'rnn',
#     'rnn_type': 'GRU',
#     'layers': 3,
#     'max_generator_batches': 2,
    'dropout': 0.3,
#     'enc_rnn_size': 512,
#     'dec_rnn_size': 512,
#     'src_word_vec_size': 512,
#     'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 128,
    'train_steps': 20000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 5,
    'early_stop_threshold': 0.1,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-16 14:53:42,839 INFO]  * src vocab size = 8870
[2019-02-16 14:53:42,840 INFO]  * tgt vocab size = 20071
[2019-02-16 14:53:42,842 INFO] Building model...
[2019-02-16 14:53:47,064 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out

meet early stop condition, prev best loss: 53.285713015726266 current loss: 53.31460524327256


'2a82e28e8fdf56498eba4ee34eee9ac0'

In [17]:
model_id = '2a82e28e8fdf56498eba4ee34eee9ac0'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-16 15:11:53,568 INFO] Translating shard 0.
[2019-02-16 15:13:34,420 INFO] Translating shard 1.


PRED AVG SCORE: -0.7467, PRED PPL: 2.1100
GOLD AVG SCORE: -10.3315, GOLD PPL: 30683.9296


[2019-02-16 15:15:21,707 INFO] Translating shard 2.


PRED AVG SCORE: -0.7608, PRED PPL: 2.1400
GOLD AVG SCORE: -10.3291, GOLD PPL: 30610.4227
PRED AVG SCORE: -0.7843, PRED PPL: 2.1910
GOLD AVG SCORE: -10.2622, GOLD PPL: 28629.7671


In [18]:
evaluate_human('2a82e28e8fdf56498eba4ee34eee9ac0')

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_2a82e28e8fdf56498eba4ee34eee9ac0",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_2a82e28e8fdf56498eba4ee34eee9ac0/log",
    "model_dtype": "fp32",
    "encoder_type": "brnn",
    "decoder_type": "rnn",
    "dropout": 0.3,
    "batch_size": 128,
    "train_steps": 20000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 64,
    "early_stop_round": 5,
    "early_stop_threshold": 0.1,
    "report_every": 100
}
38.0 22.36
Now , it &apos;s time to end , and as I mentioned before , of course , I have a lot of different data if you &apos;re interested , but I just had a lot of different data if you care , but I just wanted to show this basic idea about communication communication with the brain with its language , and the potential of the potential of it .

Now I just want to wrap up , and as I was mentioning earlier o

In [16]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'brnn',
    'decoder_type': 'rnn',
#     'rnn_type': 'GRU',
    'layers': 3,
#     'max_generator_batches': 2,
    'dropout': 0.3,
    'enc_rnn_size': 512,
    'dec_rnn_size': 512,
    'src_word_vec_size': 512,
    'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 128,
    'train_steps': 20000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 5,
    'early_stop_threshold': 0.01,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-16 15:25:23,776 INFO]  * src vocab size = 8870
[2019-02-16 15:25:23,779 INFO]  * tgt vocab size = 20071
[2019-02-16 15:25:23,779 INFO] Building model...
[2019-02-16 15:25:27,407 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 512, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(512, 256, num_layers=3, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 512, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1024, 512)
        (1): LSTMCell(512, 512)
        (2): LSTMCell(512, 512)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=512, out_features=51

meet early stop condition, prev best loss: 54.596531154711215 current loss: 54.21470925529172


'6082f2c966d9ea8dbb7afb565d719193'

In [17]:
model_id = '6082f2c966d9ea8dbb7afb565d719193'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-16 15:52:50,313 INFO] Translating shard 0.
[2019-02-16 15:54:43,488 INFO] Translating shard 1.


PRED AVG SCORE: -0.6527, PRED PPL: 1.9208
GOLD AVG SCORE: -10.9694, GOLD PPL: 58069.3216


[2019-02-16 15:56:40,825 INFO] Translating shard 2.


PRED AVG SCORE: -0.6641, PRED PPL: 1.9427
GOLD AVG SCORE: -10.9644, GOLD PPL: 57782.8891
PRED AVG SCORE: -0.6866, PRED PPL: 1.9870
GOLD AVG SCORE: -10.8774, GOLD PPL: 52963.2560


In [21]:
evaluate_human('6082f2c966d9ea8dbb7afb565d719193')

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_6082f2c966d9ea8dbb7afb565d719193",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_6082f2c966d9ea8dbb7afb565d719193/log",
    "model_dtype": "fp32",
    "encoder_type": "brnn",
    "decoder_type": "rnn",
    "layers": 3,
    "dropout": 0.3,
    "enc_rnn_size": 512,
    "dec_rnn_size": 512,
    "src_word_vec_size": 512,
    "tgt_word_vec_size": 512,
    "batch_size": 128,
    "train_steps": 20000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 64,
    "early_stop_round": 5,
    "early_stop_threshold": 0.01,
    "report_every": 100
}
37.86 22.47
Now I ended up , and as I mentioned earlier before , of course I have a lot of data if you care , but I just want to give you this basic idea of communication to the brain with its language , and the potential power of the possibility .

Now I just want to wrap up , and a

### Change early stop criterion to loss from acc.

In [ ]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'brnn',
    'decoder_type': 'rnn',
#     'rnn_type': 'GRU',
    'layers': 3,
#     'max_generator_batches': 2,
    'dropout': 0.3,
    'enc_rnn_size': 512,
    'dec_rnn_size': 512,
    'src_word_vec_size': 512,
    'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 128,
    'train_steps': 100000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 64,
    'early_stop_round': 10,
    'early_stop_threshold': 0,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-17 14:32:11,800 INFO]  * src vocab size = 8870
[2019-02-17 14:32:11,803 INFO]  * tgt vocab size = 20071
[2019-02-17 14:32:11,803 INFO] Building model...
[2019-02-17 14:32:15,534 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 512, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(512, 256, num_layers=3, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 512, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1024, 512)
        (1): LSTMCell(512, 512)
        (2): LSTMCell(512, 512)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=512, out_features=51

0 Testing GPU memory capability with batch shape: torch.Size([200, 128, 1]) torch.Size([190, 128, 1])


[2019-02-17 14:33:19,592 INFO] Step 100/100000; loss: 4036334.778355 acc:   8.13; ppl: 785.17; xent: 6.67; lr: 0.00100; 11879/9473 tok/s;     64 sec
[2019-02-17 14:34:06,330 INFO] Step 200/100000; loss: 2039835.030886 acc:  19.86; ppl: 219.52; xent: 5.39; lr: 0.00100; 9789/8096 tok/s;    111 sec
[2019-02-17 14:34:49,150 INFO] Step 300/100000; loss: 1385664.103316 acc:  28.66; ppl: 110.14; xent: 4.70; lr: 0.00100; 8058/6883 tok/s;    153 sec
[2019-02-17 14:35:28,084 INFO] Step 400/100000; loss: 991060.433813 acc:  34.69; ppl: 67.62; xent: 4.21; lr: 0.00100; 6953/6041 tok/s;    192 sec


In [17]:
model_id = '091ebe12a299ced931600c799f0937b5'
args = build_translate_args_reverse(model_id)
args['batch_size'] = 32
reverse_runner.translate(args, model_id)

[2019-02-17 11:47:52,429 INFO] Translating shard 0.
[2019-02-17 11:49:32,224 INFO] Translating shard 1.


PRED AVG SCORE: -0.5925, PRED PPL: 1.8085
GOLD AVG SCORE: -11.1411, GOLD PPL: 68947.5836


[2019-02-17 11:51:14,967 INFO] Translating shard 2.


PRED AVG SCORE: -0.6029, PRED PPL: 1.8274
GOLD AVG SCORE: -11.1182, GOLD PPL: 67386.9716
PRED AVG SCORE: -0.6267, PRED PPL: 1.8714
GOLD AVG SCORE: -11.0101, GOLD PPL: 60480.0534


In [18]:
model_id = '091ebe12a299ced931600c799f0937b5'
evaluate_human(model_id)

{
    "data": "data/nmt15-reverse",
    "save_model": "/mnt/drive-2t/model/opennmt_091ebe12a299ced931600c799f0937b5",
    "gpu_ranks": 0,
    "tensorboard": null,
    "tensorboard_log_dir": "/mnt/drive-2t/log/opennmt_091ebe12a299ced931600c799f0937b5/log",
    "model_dtype": "fp32",
    "encoder_type": "brnn",
    "decoder_type": "rnn",
    "layers": 3,
    "dropout": 0.3,
    "enc_rnn_size": 512,
    "dec_rnn_size": 512,
    "src_word_vec_size": 512,
    "tgt_word_vec_size": 512,
    "batch_size": 64,
    "train_steps": 100000,
    "optim": "adam",
    "learning_rate": 0.001,
    "valid_step": 500,
    "valid_batch": 32,
    "early_stop_round": 5,
    "early_stop_threshold": 0.01,
    "report_every": 100
}
37.51 22.73
Now , it &apos;s time that I end up , and as I mentioned earlier , of course , I have a lot of data if you care , but I just want to show this fundamental idea of communication to the brain in its language , and the potential power of that capacity .

Now I just want to w

In [16]:
model_args = {
    'model_dtype': 'fp32',
    'encoder_type': 'rnn',
    'decoder_type': 'rnn',
#     'rnn_type': 'GRU',
#     'layers': 3,
#     'max_generator_batches': 2,
    'dropout': 0.3,
#     'enc_rnn_size': 512,
#     'dec_rnn_size': 512,
#     'src_word_vec_size': 512,
#     'tgt_word_vec_size': 512
}
train_args = {
    'batch_size': 64,
    'train_steps': 100000,
    'optim': 'adam',
    'learning_rate': 0.001,
    'valid_step': 500,
    'valid_batch': 32,
    'early_stop_round': 5,
    'early_stop_threshold': 0.1,
    'report_every': 100
}

total_args = {}
total_args.update(model_args)
total_args.update(train_args)

reverse_runner.run(total_args)
reverse_runner.model_id

[2019-02-17 00:21:24,263 INFO]  * src vocab size = 8870
[2019-02-17 00:21:24,264 INFO]  * tgt vocab size = 20071
[2019-02-17 00:21:24,266 INFO] Building model...
[2019-02-17 00:21:28,526 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8870, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(20071, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out): Linear(in_feature

last accent valid acc: -inf


[2019-02-17 00:23:47,013 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654
[2019-02-17 00:24:08,988 INFO] Validation perplexity: 103.056
[2019-02-17 00:24:08,989 INFO] Validation accuracy: 28.6024
[2019-02-17 00:24:38,637 INFO] Step 600/100000; loss: 630023.420390 acc:  29.17; ppl: 107.71; xent: 4.68; lr: 0.00100; 3046/2589 tok/s;    190 sec
[2019-02-17 00:25:08,394 INFO] Step 700/100000; loss: 602951.637411 acc:  31.57; ppl: 88.65; xent: 4.48; lr: 0.00100; 5284/4518 tok/s;    220 sec
[2019-02-17 00:25:38,678 INFO] Step 800/100000; loss: 591457.869202 acc:  33.23; ppl: 77.78; xent: 4.35; lr: 0.00100; 5279/4486 tok/s;    250 sec
[2019-02-17 00:26:09,433 INFO] Step 900/100000; loss: 565543.698792 acc:  35.44; ppl: 64.40; xent: 4.17; lr: 0.00100; 5207/4415 tok/s;    281 sec
[2019-02-17 00:26:40,538 INFO] Step 1000/100000; loss: 542394.963114 acc:  36.82; ppl: 56.35; xent: 4.03; lr: 0.00100; 5059/4325 tok/s;    312 sec
[2019-02-17 00:26:40,646 INFO] Loadin

last accent valid acc: -1042816.4230957031


[2019-02-17 00:27:03,315 INFO] Validation perplexity: 45.7972
[2019-02-17 00:27:03,315 INFO] Validation accuracy: 38.9023
[2019-02-17 00:27:34,912 INFO] Step 1100/100000; loss: 535783.966767 acc:  37.60; ppl: 52.39; xent: 3.96; lr: 0.00100; 2914/2489 tok/s;    366 sec
[2019-02-17 00:28:06,741 INFO] Step 1200/100000; loss: 520141.352391 acc:  39.06; ppl: 46.32; xent: 3.84; lr: 0.00100; 5013/4261 tok/s;    398 sec
[2019-02-17 00:28:38,868 INFO] Step 1300/100000; loss: 502548.408494 acc:  40.11; ppl: 41.99; xent: 3.74; lr: 0.00100; 4902/4185 tok/s;    430 sec
[2019-02-17 00:29:11,669 INFO] Step 1400/100000; loss: 501804.743335 acc:  40.71; ppl: 39.98; xent: 3.69; lr: 0.00100; 4883/4148 tok/s;    463 sec
[2019-02-17 00:29:45,750 INFO] Loading dataset from data/nmt15-reverse.train.0.pt, number of examples: 95844
[2019-02-17 00:29:46,645 INFO] Step 1500/100000; loss: 482832.172116 acc:  41.61; ppl: 36.18; xent: 3.59; lr: 0.00100; 4517/3847 tok/s;    498 sec
[2019-02-17 00:29:46,749 INFO] Loa

...

[2019-02-17 01:26:47,803 INFO] Step 12600/100000; loss: 252385.248033 acc:  60.57; ppl:  6.46; xent: 1.87; lr: 0.00100; 4683/3944 tok/s;   3919 sec
[2019-02-17 01:26:58,081 INFO] Step 12700/100000; loss: 237192.034684 acc:  61.37; ppl:  6.09; xent: 1.81; lr: 0.00100; 14829/12778 tok/s;   3929 sec
[2019-02-17 01:27:09,279 INFO] Step 12800/100000; loss: 257974.215482 acc:  60.85; ppl:  6.23; xent: 1.83; lr: 0.00100; 14897/12595 tok/s;   3941 sec
[2019-02-17 01:27:19,766 INFO] Step 12900/100000; loss: 224642.598003 acc:  62.25; ppl:  5.74; xent: 1.75; lr: 0.00100; 14339/12264 tok/s;   3951 sec
[2019-02-17 01:27:30,481 INFO] Step 13000/100000; loss: 236886.305157 acc:  61.52; ppl:  5.98; xent: 1.79; lr: 0.00100; 14452/12361 tok/s;   3962 sec
[2019-02-17 01:27:30,578 INFO] Loading dataset from data/nmt15-reverse.valid.0.pt, number of examples: 10654


last accent valid acc: -560795.9261474609


[2019-02-17 01:27:53,652 INFO] Validation perplexity: 12.3892
[2019-02-17 01:27:53,653 INFO] Validation accuracy: 54.1907
[2019-02-17 01:27:53,654 INFO] Saving checkpoint /mnt/drive-2t/model/opennmt_153a72b0efdd37e8921bffa8bd6cc110_step_11000.pt


Decent round: 5
meet early stop condition, prev best loss: -560795.9261474609 current loss: -566220.6053466797


'153a72b0efdd37e8921bffa8bd6cc110'